In [269]:
import numpy as np # linear algerbra
import pandas as pd # data processing

In [270]:
data = pd.read_csv('data/Train_data.csv', error_bad_lines = False)
data.head(5)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,8253,Little Town on the Prairie (Little House #7),Laura Ingalls Wilder/Garth Williams,4.19,0060885432,9780060885434,eng,374,71853,646,1/1/2007,HarperTrophy
1,11101,First Comes Love (Hot Water California #1),Christie Ridgway,3.81,0380818957,9780380818952,en-US,384,1051,47,1/1/2002,Avon Books
2,28193,When Santa Fell to Earth,Cornelia Funke/Paul Howard/Oliver G. Latsch,3.76,043978204X,9780439782043,eng,167,1840,245,10/1/2006,Chicken House / Scholastic
3,25047,The Case of the Stinky Science Project (Jigsaw...,James Preller/John Speirs/James Preller,3.83,0439114284,9780439114288,en-US,80,162,11,3/1/2001,Scholastic Paperbacks
4,17800,Dance Dance Dance,Haruki Murakami,4.04,0099448769,9780099448761,eng,393,50169,2117,2/7/2002,Vintage


In [271]:
import nltk
# token_title = nltk.tokenize.word_tokenize(data['title'][0])
# token_title_without_symbol = [w for w in token_title if w in re.sub("[^a-zA-Z]", " ", w)]
# string_title=""
# for i in token_title_without_symbol:
#     string_title += i+" "

# # remo1 = 'Little Town on the Prairie Little House'
# print(string_title)
# print(data['title'][0].split(' '))
# data['tokenized_title'][0] = data['title'][0]
data['cleaned_title']=data['title']
for i in range(0, len(data)):
    token_title = nltk.tokenize.word_tokenize(data['title'][i])
    token_title_without_symbol = [w for w in token_title if w in re.sub("[^a-zA-Z]", " ", w)]
    string_title=""
    for j in token_title_without_symbol:
        string_title += j+" "
    string_title = string_title.lower()
    data['cleaned_title'][i] = string_title 
# data['tokenized_title'] =data['title']
# data['tokenized_title'] =data['tokenized_title'].apply(nltk.tokenize.word_tokenize())
print(data['cleaned_title'])

D:\winPython\WPy64-3740\python-3.7.4.amd64\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0                little town on the prairie little house 
1                  first comes love hot water california 
2                               when santa fell to earth 
3       the case of the stinky science project jigsaw ...
4                                      dance dance dance 
5       the tolkien companion and guide volume reader ...
6       mapping the big picture integrating curriculum...
7       the burglar who thought he was bogart bernie r...
8       the prime of miss jean brodie the girls of sle...
9                     desperation the regulators box set 
10      breaking the spell religion as a natural pheno...
11                                            prime time 
12                           i love saturdays y domingos 
13                            christmas in noisy village 
14      in search of memory the emergence of a new sci...
15                                 the wapshot chronicle 
16                        the gardens of emily dickinson 
17            

In [64]:
# from sklearn.feature_extraction.text import CountVectorizer
# corpus = data['title']

In [272]:
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# stemmer = SnowballStemmer('english')
words = stopwords.words('english')
data['cleaned_title'] = data['cleaned_title'].apply(lambda x:" ".join([i for i in re.sub("[^a-zA-Z]"," ",x).split() if i not in words]))

In [180]:
# print(data['cleaned_title'])
# print(words)
# cntVector = CountVectorizer(stop_words=stopwords)

In [273]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [i for i in data['cleaned_title']]
# print(corpus)
cntVector=CountVectorizer()
cntTF=cntVector.fit_transform(corpus)
# print(cntTF[3])

In [274]:
from sklearn.decomposition import LatentDirichletAllocation
#参数可调n_componenets是对于一个title生成几个可能的主题，max_itr迭代次数
lda = LatentDirichletAllocation(n_components =3, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
docres = lda.fit_transform(cntTF)

In [279]:
# feature_names = lda.get_feature_names(cntTF)
#  = lda.get_feature_names(cntTF)
# print(docres[0].index(max(docres[0])))
# temp = docres[0].tolist()
# print(temp.index(max(temp)))
# print(cntTF[9999][0])
# print(len(lda.components_[0]))
# print(cntVector)

#每个title会有n个可能的主题，选出中最可能的那一个主题，返回主题的index作为feature
topics_for_each_title = []
for i in range(0, len(data)):
    temp = docres[i].tolist()
    topic = temp.index(max(temp))
    topics_for_each_title.append(topic)
# print(topics_for_each_title)
#这里开始是把这个feature加到dataFrame中作为一个feature
data['title_topic'] = topics_for_each_title
print(data['title_topic'])

2


In [46]:
# import jieba
# def word_cut(mytext):
#     return " ".join(jieba.cut(mytext))

In [47]:
# data["title_cutted"] = data["title"].apply(word_cut)

In [48]:
#data["title_cutted"] = ''.join(data['title'].str.replace('(','').str.replace(')','').str.split())

In [49]:
# data["title_cutted"].head(5)

0    Little   Town   on   the   Prairie     ( Littl...
1    First   Comes   Love   ( Hot   Water     Calif...
2                     When   Santa   Fell   to   Earth
3    The   Case   of   the   Stinky   Science   Pro...
4                                Dance   Dance   Dance
Name: title_cutted, dtype: object

In [50]:
# data["title_cutted"] = data["title_cutted"].str.replace('(','').str.replace(')','')

In [51]:
# data["title_cutted"].head(5)

0    Little   Town   on   the   Prairie      Little...
1    First   Comes   Love    Hot   Water     Califo...
2                     When   Santa   Fell   to   Earth
3    The   Case   of   the   Stinky   Science   Pro...
4                                Dance   Dance   Dance
Name: title_cutted, dtype: object

In [52]:
# data["title_cutted"]

0       Little   Town   on   the   Prairie      Little...
1       First   Comes   Love    Hot   Water     Califo...
2                        When   Santa   Fell   to   Earth
3       The   Case   of   the   Stinky   Science   Pro...
4                                   Dance   Dance   Dance
5       The   J . R . R .   Tolkien   Companion   and ...
6       Mapping   the   Big   Picture :   Integrating ...
7       The   Burglar   Who   Thought   He   Was   Bog...
8       The   Prime   of   Miss   Jean   Brodie   /   ...
9        Desperation   /   The   Regulators :   Box   Set
10      Breaking   the   Spell :   Religion   as   a  ...
11                                           Prime   Time
12                    I   Love   Saturdays   y   domingos
13                       Christmas   in   Noisy   Village
14      In   Search   of   Memory :   The   Emergence ...
15                              The   Wapshot   Chronicle
16                 The   Gardens   of   Emily   Dickinson
17            

## 文本向量化

In [53]:
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# n_features = 1000
# tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
#                                 max_features=n_features,
#                                 stop_words='english',
#                                 max_df = 0.5,
#                                 min_df = 10)
# tf = tf_vectorizer.fit_transform(data["title_cutted"])

In [56]:
# print(tf[0])

  (0, 389)	2
  (0, 649)	1
  (0, 312)	1


## using LDA

In [16]:
# from sklearn.decomposition import LatentDirichletAllocation
# #每个title给10个topic，迭代50次
# lda = LatentDirichletAllocation(n_components=10, max_iter=50,
#                                 learning_method='online',
#                                 learning_offset=50.,
#                                 random_state=0)

In [33]:
# topic_for_title = lda.fit(tf)

In [35]:
# print(topic_for_title)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=50,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [18]:
# def print_top_words(model, feature_names, n_top_words):
#     for topic_idx, topic in enumerate(model.components_):
#         print("Topic #%d:" % topic_idx)
#         print(" ".join([feature_names[i]
#     for i in topic.argsort()[:-n_top_words - 1:-1]]))
#     print()

In [19]:
# n_top_words = 20
# tf_feature_names = tf_vectorizer.get_feature_names()
# print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
life great philosophy good mystery shadow introduction easy tolkien summer game century ring island inside times early boys ice paradise
Topic #1:
tales la tale castle shakespeare dreams library adventure novel case mr battle peter ghost les madness princess jones stone scarlet
Topic #2:
vol world war new art short woman long city notes alchemist ii end diary henry iv fullmetal fall 14 work
Topic #3:
guide story american death volume god best harry women essays murder potter heart books high days year valley jane writing
Topic #4:
star like wars poems selected collected big fiction 12 christmas don le theory essential blue 15 literature far discworld anthology
Topic #5:
stories time el 10 journey modern jack saga red cat biography day living code memoir child sword winter song making
Topic #6:
man trilogy dark writings family sea men young works empire old country dance shannara daughter faith laura divine shadows volume
Topic #7:
black adventures 11 power magic science moon 

In [20]:
# data["title_cutted"].head(5)

0    Little   Town   on   the   Prairie      Little...
1    First   Comes   Love    Hot   Water     Califo...
2                     When   Santa   Fell   to   Earth
3    The   Case   of   the   Stinky   Science   Pro...
4                                Dance   Dance   Dance
Name: title_cutted, dtype: object

In [21]:
# lda.transform(tf)

array([[0.02      , 0.02      , 0.02      , ..., 0.02      , 0.82      ,
        0.02      ],
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.775     ,
        0.025     ],
       [0.05      , 0.05      , 0.05      , ..., 0.05      , 0.05      ,
        0.55      ],
       ...,
       [0.05      , 0.05      , 0.55      , ..., 0.05      , 0.05      ,
        0.05      ],
       [0.03333333, 0.36666706, 0.03333333, ..., 0.03333333, 0.03333333,
        0.03333333],
       [0.03333333, 0.03333333, 0.03333333, ..., 0.03333333, 0.36666688,
        0.03333333]])

In [22]:
# temp=lda.transform(tf)
# print(len(temp))

10000


In [27]:
# print(temp[0])

[0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.82 0.02]


In [32]:
# print(len(tf_feature_names))

728
